<a href="https://colab.research.google.com/github/Davilirio/Python_data_analysis/blob/master/MQ2E.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install linearmodels

     |████████████████████████████████| 1.5MB 6.0MB/s 


In [3]:
import numpy as np
import statsmodels.api as sm
from statsmodels.formula.api import ols
from scipy.ndimage.interpolation import shift
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.sandbox.regression.gmm import IV2SLS as sls
from linearmodels.iv import IV2SLS

In [55]:
x = [9.384037		,12.49205		,10.15123		,3.969234		,11.22021		,7.053592		,11.21397		,12.92253		,5.260528		,9.245915		,12.23626		,12.62259		,10.76048		,12.52159		,8.596485		,13.82085		,7.229485		,12.59341		,8.587203		,10.61918		,6.158685		,8.334646		,14.77749		,12.51743		,9.898167		,12.00444		,13.82187		,10.57086		,9.373612		,13.00577		,10.001		,13.13393		,17.23179		,11.5966		,9.242477		,1.639814		,9.865471		,15.59969		,9.507248		,11.0872		,10.35309		,16.70251		,15.07791		,9.933088		,12.03716		,15.96111		,9.794756		,6.223539		,6.128167		,6.080409		,13.06767		,8.728737		,10.24003		,16.15474		,13.02866		,12.77594		,13.78554		,12.47706		,10.96659		,4.014013		,15.72047		,4.404097		,13.03976		,7.524463		,9.439558		,5.144634		,11.48883		,9.110517		,11.63254		,11.68001		,11.27001		,10.97662		,12.4446		,12.7265		,10.36107		,6.382313		,12.67794		,10.52474		,14.45934		,8.914623		,8.562088		,9.93448		,11.11299		,14.98805		,15.76222		,10.80198		,9.946788		,4.821705		,9.602241		,9.785084		]
len(x)

90

In [56]:
y=[		69.56599,		70.93386,		67.13956,		52.94334,		60.85608,		56.86313,		63.83839,		71.32599,		58.90408,		59.98986,		66.67406,		69.81581,		70.42963,		74.09744,		67.96977,		74.29905,		65.61272,		70.71265,		64.43282,		64.37928,		56.97442,		56.84522,		69.42366,		71.19716,		67.4888,		70.10452,		73.89326,		71.14352,		66.53094,		71.50073,	67.91754,		73.2056,		84.23586,	78.43824,		71.01487,		52.31186,		59.90692,		74.60895,		68.61565,	69.89749,		68.88191,		81.27016,		84.79838,		76.52046,		76.17368,		81.86633,		71.552,		59.48807,		52.08296,		47.71331,		59.84106,		55.49755,		58.29302,		72.74291,		72.32827,		71.89715,		74.87852,		73.12127,		69.03547,		52.41491,		68.21809,		52.92928,		62.25097,		58.08406,		59.27546,		52.33179,		62.58669,		62.57345,		68.40366,		71.4606,		73.24609,		73.09506,	74.18145,	75.85994,		72.53261,		63.16094,		70.98943,		70.26633,		75.91901,		65.85706,		60.62146,	59.54728,		61.82454,		70.09711,		76.07267,		68.19608,		63.16901,		50.94413,		55.95074,		58.97906]
len(y)

90

In [38]:
def mq2e(y, x):
  
  df = pd.DataFrame()
  
  # criando as colunas para o minimo quadrado em 2 estagios
  df['x'] = x
  df['x_lag'] = df['x'].shift(1)
  df['y'] = y
  df['y_lag'] = df['y'].shift(1)
  df['Intercept'] = 1

  mq2e = IV2SLS(dependent=df['y'],
            exog=df[['Intercept','x']],
            endog=df['y_lag'],
            instruments=df['x_lag'])

  mq2e_fitted = mq2e.fit()

  reg = ols(formula = 'y ~ y_lag + x', data = df)
  OLS = reg.fit()

  ols_params = OLS.params
  mq2e_params = mq2e_fitted.params

  bias = dict()

  bias['vies_beta1'] = ols_params['Intercept'] - mq2e_params['Intercept']
  bias['vies_beta2'] = ols_params['y_lag'] - mq2e_params['y_lag']
  bias['vies_beta3'] = ols_params['x'] - mq2e_params['x']

  # print(OLS.)

  return mq2e_fitted.summary, OLS.summary(), bias, df

In [57]:
iv2sls, mqo, vies, df = mq2e(y, x)

/usr/local/lib/python3.6/dist-packages/linearmodels/iv/data.py:25: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if is_categorical(s):
/usr/local/lib/python3.6/dist-packages/linearmodels/utility.py:549: MissingValueWarning: 
Inputs contain missing values. Dropping rows with missing observations.
  warnings.warn(missing_value_warning_msg, MissingValueWarning)


In [58]:
mqo

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.965
Model:                            OLS   Adj. R-squared:                  0.964
Method:                 Least Squares   F-statistic:                     1179.
Date:                Thu, 17 Dec 2020   Prob (F-statistic):           3.08e-63
Time:                        21:05:00   Log-Likelihood:                -162.52
No. Observations:                  89   AIC:                             331.0
Df Residuals:                      86   BIC:                             338.5
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      7.6972      1.465      5.255      0.000       4.785      10.609
y_lag          0.5581      0.020     27.457      0.000       0.518       0.599
x              2.0353      0.052     39.394      0.000       1.933       2.138
==============================================================================
Omnibus:                        3.582   Durbin-Watson:                   0.477
Prob(Omnibus):                  0.167   Jarque-Bera (JB):                3.114
Skew:                          -0.362   Prob(JB):                        0.211
Kurtosis:                       2.439   Cond. No.                         617.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [59]:
iv2sls

<class 'linearmodels.compat.statsmodels.Summary'>
"""
                          IV-2SLS Estimation Summary                          
==============================================================================
Dep. Variable:                      y   R-squared:                      0.9620
Estimator:                    IV-2SLS   Adj. R-squared:                 0.9611
No. Observations:                  89   F-statistic:                    1913.3
Date:                Thu, Dec 17 2020   P-value (F-stat)                0.0000
Time:                        21:05:00   Distribution:                  chi2(2)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Intercept      11.267     1.9203     5.8670     0.0000      7.5029      15.030
x              2.0386     0.0513     39.766     0.0000      1.9381      2.1391
y_lag          0.5043     0.0277     18.214     0.0000      0.4500      0.5585
==============================================================================

Endogenous: y_lag
Instruments: x_lag
Robust Covariance (Heteroskedastic)
Debiased: False
"""

In [60]:
vies

{'vies_beta1': -3.5694568892245826,
 'vies_beta2': 0.053861175368693726,
 'vies_beta3': -0.0032797329225884297}

## TESTE

In [61]:
olss=ols(formula=('y~y_lag+x'), data=df)
reg = olss.fit()

In [62]:
reg.summary2()

<class 'statsmodels.iolib.summary2.Summary'>
"""
                 Results: Ordinary least squares
=================================================================
Model:              OLS              Adj. R-squared:     0.964   
Dependent Variable: y                AIC:                331.0348
Date:               2020-12-17 21:05 BIC:                338.5007
No. Observations:   89               Log-Likelihood:     -162.52 
Df Model:           2                F-statistic:        1179.   
Df Residuals:       86               Prob (F-statistic): 3.08e-63
R-squared:          0.965            Scale:              2.3361  
-------------------------------------------------------------------
            Coef.    Std.Err.      t      P>|t|    [0.025    0.975]
-------------------------------------------------------------------
Intercept   7.6972     1.4649    5.2546   0.0000   4.7852   10.6092
y_lag       0.5581     0.0203   27.4574   0.0000   0.5177    0.5986
x           2.0353     0.0517   39.3936   0.0000   1.9326    2.1380
-----------------------------------------------------------------
Omnibus:               3.582        Durbin-Watson:          0.477
Prob(Omnibus):         0.167        Jarque-Bera (JB):       3.114
Skew:                  -0.362       Prob(JB):               0.211
Kurtosis:              2.439        Condition No.:          617  
=================================================================

"""